# Configuração de Ambiente dbt com DuckDB

Este notebook automatiza a criação e configuração de um ambiente `dbt` com `DuckDB` usando ambiente virtual Python.

**Etapas:**
1. Criação de ambiente virtual
2. Instalação de dependências
3. Criação de banco DuckDB
4. Inicialização de projeto dbt
5. Criação do arquivo de perfil do dbt
6. Teste final da configuração

## 0- Imports gerais

In [18]:
%pip install -q duckdb
%pip install -q dbt-duckdb

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [8]:
import os
import sys
import subprocess
import venv
import json
from pathlib import Path
import duckdb

## 1. Criação de ambiente virtual

In [9]:
def create_and_activate_venv():
    venv_name = ".venv"
    if os.path.exists(venv_name):
        print(f"✅ Ambiente virtual '{venv_name}' já existe.")
    else:
        print(f"🔧 Criando ambiente virtual '{venv_name}'...")
        venv.create(venv_name, with_pip=True)
        print("✅ Ambiente virtual criado com sucesso!")

    if sys.platform == "win32":
        python_path = os.path.join(venv_name, "Scripts", "python.exe")
        pip_path = os.path.join(venv_name, "Scripts", "pip.exe")
    else:
        python_path = os.path.join(venv_name, "bin", "python")
        pip_path = os.path.join(venv_name, "bin", "pip")
    return python_path, pip_path

python_path, pip_path = create_and_activate_venv()

🔧 Criando ambiente virtual '.venv'...
✅ Ambiente virtual criado com sucesso!


## 2. Instalação de dependências

In [10]:
def install_packages(pip_path):
    packages = ["dbt-duckdb", "pandas", "duckdb"]
    for package in packages:
        print(f"📦 Instalando {package}...")
        subprocess.run([pip_path, "install", package], check=True)

install_packages(pip_path)

📦 Instalando dbt-duckdb...
  Using cached duckdb-1.4.2-cp312-cp312-manylinux_2_26_x86_64.manylinux_2_28_x86_64.whl.metadata (4.3 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.4/114.4 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.4/87.4 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 986.2/986.2 kB 27.0 MB/s eta 0:00:0000:01
Using cached duckdb-1.4.2-cp312-cp312-manylinux_2_26_x86_64.manylinux_2_28_x86_64.whl (20.5 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.3/108.3 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.9/144.9 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: /workspaces/Atv_DBT_Introducao/.venv/bin/python -m pip install --upgrade pip


📦 Instalando pandas...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 40.2 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 31.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 9.4 MB/s eta 0:00:00ta 0:00:01
📦 Instalando duckdb...



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: /workspaces/Atv_DBT_Introducao/.venv/bin/python -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: /workspaces/Atv_DBT_Introducao/.venv/bin/python -m pip install --upgrade pip


### 3. Inicialização de projeto dbt

```
dbt init meu_projeto_dbt
```

In [11]:
def create_model_directories():
    base_path = "meu_projeto_dbt/models"
    directories = ["raw", "staging", "intermediate", "mart"]
    for d in directories:
        dir_path = os.path.join(base_path, d)
        os.makedirs(dir_path, exist_ok=True)
        init_file = os.path.join(dir_path, "__init__.py")
        if not os.path.exists(init_file):
            with open(init_file, 'w') as f:
                f.write(f"# {d} models\n")
create_model_directories()

### 4. Inicialização de projeto dbt

In [12]:

def create_duckdb_database():
    db_path = "bd/dev.duckdb"
    os.makedirs(os.path.dirname(db_path), exist_ok=True)
    conn = duckdb.connect(db_path)
    result = conn.execute("SELECT 'Conexão estabelecida com sucesso!' as status").fetchone()
    print(f"🗄️ Banco DuckDB criado/verificado: {db_path}")
    print(f"📊 Status: {result[0]}")
    conn.close()
    return db_path
    # Criação dos schemas no banco DuckDB
  

create_duckdb_database()

🗄️ Banco DuckDB criado/verificado: bd/dev.duckdb
📊 Status: Conexão estabelecida com sucesso!


'bd/dev.duckdb'

#### 4.1 Teste de conexão ao banco

In [13]:
def test_database_connection():
    conn = duckdb.connect("bd/dev.duckdb")
    result = conn.execute("SELECT version() as duckdb_version").fetchone()
    print(f"🔌 Conexão OK - Versão DuckDB: {result[0]}")
    conn.execute("CREATE TABLE IF NOT EXISTS teste_conexao (id INTEGER, mensagem VARCHAR)")
    conn.execute("INSERT INTO teste_conexao VALUES (1, 'Teste ok')")
    print(conn.execute("SELECT * FROM teste_conexao").fetchall())
    conn.execute("DROP TABLE teste_conexao")
    conn.close()
test_database_connection()

🔌 Conexão OK - Versão DuckDB: v1.4.2
[(1, 'Teste ok')]


###  5. Criação do arquivo de perfil do dbt

In [14]:
try:
    import yaml
except ImportError:
    yaml = None

def create_dbt_profile():
    profile_dir = os.path.expanduser("~/.dbt")
    os.makedirs(profile_dir, exist_ok=True)
    profiles_path = os.path.join(profile_dir, "profiles.yml")
    config = {
        "meu_projeto_dbt": {
            "target": "dev",
            "outputs": {
                "dev": {
                    "type": "duckdb",
                    "path": os.path.abspath("bd/dev.duckdb"),
                    "schema": "main"
                }
            }
        }
    }
    with open(profiles_path, 'w') as f:
        if yaml:
            yaml.dump(config, f)
        else:
            json.dump(config, f, indent=2)
    print(f"✅ Perfil dbt criado em: {profiles_path}")
create_dbt_profile()

✅ Perfil dbt criado em: /home/codespace/.dbt/profiles.yml


### 6. Teste final da configuração

In [15]:
def test_dbt_configuration():
    os.chdir("meu_projeto_dbt")
    dbt_path = os.path.join("..", ".venv", "Scripts" if sys.platform == "win32" else "bin", "dbt")
    result = subprocess.run([dbt_path, "debug"], capture_output=True, text=True)
    print(result.stdout)
    os.chdir("..")
test_dbt_configuration()

16:15:50  Running with dbt=1.10.15
16:15:50  dbt version: 1.10.15
16:15:50  python version: 3.12.3
16:15:50  python path: /workspaces/Atv_DBT_Introducao/.venv/bin/python
16:15:50  os info: Linux-6.8.0-1030-azure-x86_64-with-glibc2.39
16:15:51  Using profiles dir at /home/codespace/.dbt
16:15:51  Using profiles.yml file at /home/codespace/.dbt/profiles.yml
16:15:51  Using dbt_project.yml file at /workspaces/Atv_DBT_Introducao/meu_projeto_dbt/dbt_project.yml
16:15:51  adapter type: duckdb
16:15:51  adapter version: 1.10.0
16:15:51  Configuration:
16:15:51    profiles.yml file [OK found and valid]
16:15:51    dbt_project.yml file [ERROR not found]
16:15:51  Required dependencies:
16:15:51   - git [OK found]

16:15:51  Connection:
16:15:51    database: dev
16:15:51    schema: main
16:15:51    path: /workspaces/Atv_DBT_Introducao/bd/dev.duckdb
16:15:51    config_options: None
16:15:51    extensions: None
16:15:51    settings: {}
16:15:51    external_root: .
16:15:51    use_credential_provid